# 사용자 정의 데이터셋 
- pytorch에서 딥러닝 시 대량의 데이터 사용에 따른 부하(HW, SW) 및 많은 시간 소요에 대한 해결책으로 제시 
- 대량 데이터셋 전용 처리 모듈 제공 
- Dataset과 DataLoader 
    - Dataset =>  데이터셋 전처리, 텐서화 등의 작업 진행 
    - DataLoaer => 데이터셋 인스턴스를 사용해서 배치크기 만큼 데이터 추출  

## [1] 모듈 로딩 및 데이터 준비 

In [40]:
# 모듈 로딩
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

import pandas as pd 
from sklearn.preprocessing import LabelEncoder  # 타겟 컬럼 수치화 

In [41]:
DATA_FILE = '../DATA/iris.csv'

iris_df = pd.read_csv(DATA_FILE)
iris_df.head()

,sepal.length,sepal.width,petal.length,petal.width,variety
0,5.1,3.5,1.4,0.2,Setosa
1,4.9,3.0,1.4,0.2,Setosa
2,4.7,3.2,1.3,0.2,Setosa
3,4.6,3.1,1.5,0.2,Setosa
4,5.0,3.6,1.4,0.2,Setosa


In [42]:
# 타겟 컬러 수치화 => LabelEncoder # 왜에? 원 핫 안 써?? 순서 있어도 되나?!?

encoder = LabelEncoder()
encoder.fit(iris_df['variety'])
iris_df['variety'] = encoder.transform(iris_df['variety'])

In [43]:
iris_df.head()

,sepal.length,sepal.width,petal.length,petal.width,variety
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


## [2] 사용자 정의 데이터 셋 클래스 생성 

In [44]:
# ---------------------------------------------
# 클래스 목적 : 학습용 데이터셋 텐서화 및 전처리 
# 클래스 이름 : CustomDataset
# 부모 클래스 : torch.utils.data.Dataset 
# 매개변수 : feature_df, target_df 
# ---------------------------------------------

In [45]:
class CustomDataset(Dataset):
    
    # 데이터 로딩 및 전처리 진행과 인스턴스 생성 메서드 
    def __init__(self, feature_df, target_df):
        super().__init__()

        self.feature_df = feature_df
        self.target_df = target_df
        self.n_rows = feature_df.shape[0]
        self.n_features = feature_df.shape[1] 
         
    # 데이터의 개수 반환 메서드 
    def __len__(self):
        return self.n_rows
    
    # 특정 index의 데이터와 타겟 반환 메서드 => Tensor반환 
    def __getitem__(self,idx):
        feature_ts = torch.FloatTensor(self.feature_df.iloc[idx].values)
        target_ts = torch.FloatTensor(self.target_df.iloc[idx].values)
        return feature_ts, target_ts

## [3] 데이터셋 인스턴스 생성

In [46]:
feature_df , target_df = iris_df[iris_df.columns[:-1]], iris_df[[iris_df.columns[-1]]]
print(f'feature_df =? {feature_df.shape}, target_df => {target_df.shape}')

feature_df =? (150, 4), target_df => (150, 1)


In [47]:
# iris 데이터셋 인스턴스 생성 
iris_ds = CustomDataset(feature_df, target_df)

In [48]:
iris_ds.feature_df

,sepal.length,sepal.width,petal.length,petal.width
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2
...,...,...,...,...
145,6.7,3.0,5.2,2.3
146,6.3,2.5,5.0,1.9
147,6.5,3.0,5.2,2.0
148,6.2,3.4,5.4,2.3


In [49]:
# iris 데이터셋 속성 
iris_ds.n_features, iris_ds.n_rows

(4, 150)

In [50]:
# iris 데이터셋 메서드
iris_ds[0]

(tensor([5.1000, 3.5000, 1.4000, 0.2000]), tensor([0.]))

## [4] 데이터로더 인스턴스 생성 

In [51]:
# 필요한 것 : DataSet 인스턴스, Batch_size = 1 (default) 
iris_dl = DataLoader(iris_ds)

In [52]:
iris_dl # 반복기능 있음 (__iter__() -> for문 돌려도 ㅇ )

In [53]:
iris_dl.__iter__()

In [54]:
for _ in iris_dl:
    print(_)
    break  # break 있어서 하나만 뜸 

[tensor([[5.1000, 3.5000, 1.4000, 0.2000]]), tensor([[0.]])]


In [55]:
iris_dl = DataLoader(iris_ds, batch_size=5)

In [56]:
for _ in iris_dl:
    print(_)
    break  # break 있어서 하나만 뜸 

[tensor([[5.1000, 3.5000, 1.4000, 0.2000],
        [4.9000, 3.0000, 1.4000, 0.2000],
        [4.7000, 3.2000, 1.3000, 0.2000],
        [4.6000, 3.1000, 1.5000, 0.2000],
        [5.0000, 3.6000, 1.4000, 0.2000]]), tensor([[0.],
        [0.],
        [0.],
        [0.],
        [0.]])]


batch_size 의 크기에 따라 몇 개 뺼지 정해짐 
- 일반적으로 성능 좋은 것이 32개 

In [57]:
for data_ts, target_ts in iris_dl:
    print(data_ts.shape, target_ts.shape)
    break

torch.Size([5, 4]) torch.Size([5, 1])


위의 방식대로 하면 EPOCH 사용할 필요 X 

In [58]:
# ------------------------------------
# 클래스 기능 : 파일 확장자별 데이터 프레임 변환 기능 
# 클래스 이름 : convertDataFrame
# 매개 변수 : data_path 파일 경로 
# 함수 결과 : DataFrame 
# ------------------------------------

def convertDataFrame(data_path, exit_header=0):
    ext = data_path.rsplit('.')[-1] # 뒤에서 부터 split 
    if ext == 'csv':
        return pd.read_csv(data_path, header = exit_header)
    elif ext == 'json':
        return pd.read_json(data_path, header = exit_header)
    elif ext in ['xlsx','xls']:
        return pd.read_excel(data_path, header = exit_header)
    else:
        return pd.read_table(data_path, header = exit_header)

In [60]:
# ------------------------------------
# 클래스 기능 : 파일 기반 데이터셋 
# 클래스 이름 : FileDataset
# 매개 변수 : data_path 파일 경로 
# 부모 클래스 : utils.data.Dataset 
# ------------------------------------

class FileDataset(Dataset):
    
    # 데이터 로딩 및 전처리 진행과 인스턴스 생성 메서드 
    def __init__(self, data_path):
        super().__init__()
        # 데이터 파일 => DF 변환 
        data_df = convertDataFrame(data_path)

        # DF => 피쳐와 타겟 추출 
        self.feature_df = data_df[data_df.columns[:-1]]
        self.target_df = data_df[data_df.columns[-1:]]
        
        self.n_features = data_df.shape[1]
        self.n_rows = self.feature_df.shape[0]
             
    # 데이터의 개수 반환 메서드 
    def __len__(self):
        pass
    
    # 특정 index의 데이터와 타겟 반환 메서드 => Tensor반환 
    def __getitem__(self,idx):
        feature_ts = torch.FloatTensor(self.feature_df.iloc[idx].values)
        target_ts = torch.FloatTensor(self.target_df.iloc[idx].values)
        return feature_ts, target_ts